## Data source

https://open.toronto.ca/dataset/bike-share-toronto-ridership-data/

In [1]:
import numpy as np
import pandas as pd
import datetime
import time
import requests
import json
from geopy import distance

Load Jan 2021 dataset:

In [2]:
df01 = pd.read_csv("bike-share-ridership-2021-01.csv",encoding='cp1252')
df01.head()

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
0,10644218,1315,7021,01/01/2021 00:04,Bay St / Albert St,7164.0,01/01/2021 00:26,Gould St / Yonge St (Ryerson University),6795.0,Annual Member
1,10644220,396,7534,01/01/2021 00:07,Walnut Ave / Queen St W,7524.0,01/01/2021 00:13,Lisgar Park,4176.0,Casual Member
2,10644221,86,7162,01/01/2021 00:10,Hayter St / Laplante Ave,7006.0,01/01/2021 00:11,Bay St / College St (East Side),1814.0,Annual Member
3,10644222,741,7003,01/01/2021 00:10,Madison Ave / Bloor St W,7272.0,01/01/2021 00:22,Yonge St / Dundonald St - SMART,198.0,Casual Member
4,10644223,2073,7562,01/01/2021 00:11,Priscilla Ave / Dundas St W - SMART,7562.0,01/01/2021 00:45,Priscilla Ave / Dundas St W - SMART,6688.0,Casual Member


According to data dictionary:

- Trip Id: Object ID, unique integer
- Trip Duration: Duration of a trip in seconds
- Start Station Id: ID of the station where the trip started (origin)
- Start time: mm/dd/yyyy 00:00
- Start Station Name
- End Station Id: ID of the station where the trip ended (destination)
- End Time
- End Station Name
- Bike Id
- User type: The type of user that took the trip, member (annual pass holder) or casual (24 or 72 hour pass holders)

In [3]:
df01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86371 entries, 0 to 86370
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Trip Id             86371 non-null  int64  
 1   Trip  Duration      86371 non-null  int64  
 2   Start Station Id    86371 non-null  int64  
 3   Start Time          86371 non-null  object 
 4   Start Station Name  86069 non-null  object 
 5   End Station Id      86058 non-null  float64
 6   End Time            86371 non-null  object 
 7   End Station Name    85759 non-null  object 
 8   Bike Id             86168 non-null  float64
 9   User Type           86371 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 6.6+ MB


Since the datasets are large, we decide to explore Jan 2021 data first, then apply the same workflow to other months, and finally combine 12 months together.

## Data Cleaning

### Check missing values

From the summary above, it shows null value in "End Station Name", but not all of them miss End Station Id, so we check the rows without End Station Id first

In [4]:
df01[df01["End Station Id"].isna()]

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
333,10644662,0,7470,01/01/2021 09:41,York St / Lake Shore Blvd W,NaN,01/01/2021 09:41,NaN,5761.0,Annual Member
1304,10645786,0,7063,01/01/2021 16:00,Queen's Park / Bloor St W,NaN,01/01/2021 16:00,NaN,6731.0,Annual Member
1335,10645823,0,7419,01/01/2021 16:10,Bloor St W / Huron St,NaN,01/01/2021 16:10,NaN,6835.0,Casual Member
1444,10645959,0,7281,01/01/2021 16:38,Charles St W / Balmuto St - SMART,NaN,01/01/2021 16:38,NaN,5748.0,Annual Member
1878,10646666,1336,7203,01/02/2021 09:12,Bathurst St/Queens Quay(Billy Bishop Airport),NaN,01/02/2021 09:34,NaN,2598.0,Annual Member
...,...,...,...,...,...,...,...,...,...,...
82325,10743730,0,7177,01/29/2021 20:23,East Liberty St / Pirandello St,NaN,01/29/2021 20:23,NaN,3878.0,Annual Member
83166,10744707,0,7040,01/30/2021 12:52,Euclid Ave / Bloor St W,NaN,01/30/2021 12:52,NaN,1672.0,Casual Member
83260,10744806,0,7627,01/30/2021 13:18,Shaw St / Essex St,NaN,01/30/2021 13:18,NaN,3262.0,Annual Member
83823,10745412,0,7518,01/30/2021 15:21,Lake Shore Blvd W / Colborne Lodge Dr,NaN,01/30/2021 15:21,NaN,1541.0,Annual Member


Here we find entries that have Trip Duration = 0. These might be some errors or technical issues when the bikes were released. 

In [5]:
df01[df01["Trip  Duration"] == 0]

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
333,10644662,0,7470,01/01/2021 09:41,York St / Lake Shore Blvd W,NaN,01/01/2021 09:41,NaN,5761.0,Annual Member
1304,10645786,0,7063,01/01/2021 16:00,Queen's Park / Bloor St W,NaN,01/01/2021 16:00,NaN,6731.0,Annual Member
1335,10645823,0,7419,01/01/2021 16:10,Bloor St W / Huron St,NaN,01/01/2021 16:10,NaN,6835.0,Casual Member
1444,10645959,0,7281,01/01/2021 16:38,Charles St W / Balmuto St - SMART,NaN,01/01/2021 16:38,NaN,5748.0,Annual Member
1933,10646723,0,7072,01/02/2021 10:10,Fleet St / Bathurst St,7569.0,01/02/2021 10:10,Toronto Inukshuk Park,2306.0,Annual Member
...,...,...,...,...,...,...,...,...,...,...
82862,10744378,0,7270,01/30/2021 10:44,Church St / Dundas St E - SMART,7001.0,01/30/2021 10:44,Lower Jarvis St / The Esplanade,1753.0,Annual Member
83166,10744707,0,7040,01/30/2021 12:52,Euclid Ave / Bloor St W,NaN,01/30/2021 12:52,NaN,1672.0,Casual Member
83260,10744806,0,7627,01/30/2021 13:18,Shaw St / Essex St,NaN,01/30/2021 13:18,NaN,3262.0,Annual Member
83823,10745412,0,7518,01/30/2021 15:21,Lake Shore Blvd W / Colborne Lodge Dr,NaN,01/30/2021 15:21,NaN,1541.0,Annual Member


In [6]:
df01_v1 = df01[df01["Trip  Duration"] != 0]
df01_v1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85857 entries, 0 to 86370
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Trip Id             85857 non-null  int64  
 1   Trip  Duration      85857 non-null  int64  
 2   Start Station Id    85857 non-null  int64  
 3   Start Time          85857 non-null  object 
 4   Start Station Name  85555 non-null  object 
 5   End Station Id      85830 non-null  float64
 6   End Time            85857 non-null  object 
 7   End Station Name    85533 non-null  object 
 8   Bike Id             85654 non-null  float64
 9   User Type           85857 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 7.2+ MB


In [7]:
df01_v1 = df01_v1[df01_v1["End Station Id"].notnull()]
df01_v1 = df01_v1[df01_v1["Bike Id"].notnull()]
df01_v1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85627 entries, 0 to 86370
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Trip Id             85627 non-null  int64  
 1   Trip  Duration      85627 non-null  int64  
 2   Start Station Id    85627 non-null  int64  
 3   Start Time          85627 non-null  object 
 4   Start Station Name  85326 non-null  object 
 5   End Station Id      85627 non-null  float64
 6   End Time            85627 non-null  object 
 7   End Station Name    85331 non-null  object 
 8   Bike Id             85627 non-null  float64
 9   User Type           85627 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 7.2+ MB


### Convert data type

Start and End time can be converted into date format for easier extraction and calculation later (this can also be done by string slicing, so the conversion is optional)

In [8]:
df01_v1['Start Time'] = pd.to_datetime(df01_v1['Start Time'])
df01_v1['End Time'] = pd.to_datetime(df01_v1['End Time'])

In [9]:
df01_v1.groupby(df01_v1['End Time'].dt.day_name()).count()

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type
End Time,,,,,,,,,,
Friday,13372,13372,13372,13372,13318,13372,13372,13317,13372,13372
Monday,12225,12225,12225,12225,12185,12225,12225,12183,12225,12225
Saturday,15533,15533,15533,15533,15481,15533,15533,15488,15533,15533
Sunday,12268,12268,12268,12268,12224,12268,12268,12227,12268,12268
Thursday,11922,11922,11922,11922,11878,11922,11922,11877,11922,11922
Tuesday,9482,9482,9482,9482,9447,9482,9482,9445,9482,9482
Wednesday,10825,10825,10825,10825,10793,10825,10825,10794,10825,10825


All the IDs should be string

In [10]:
df01_v1['Trip Id'] = df01_v1['Trip Id'].astype("string")
df01_v1['Start Station Id'] = df01_v1['Start Station Id'].astype("string")
df01_v1['End Station Id'] = df01_v1['End Station Id'].astype("int")
df01_v1['End Station Id'] = df01_v1['End Station Id'].astype("string")
df01_v1['Bike Id'] = df01_v1['Bike Id'].astype("int")
df01_v1['Bike Id'] = df01_v1['Bike Id'].astype("string")

df01_v1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85627 entries, 0 to 86370
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Trip Id             85627 non-null  string        
 1   Trip  Duration      85627 non-null  int64         
 2   Start Station Id    85627 non-null  string        
 3   Start Time          85627 non-null  datetime64[ns]
 4   Start Station Name  85326 non-null  object        
 5   End Station Id      85627 non-null  string        
 6   End Time            85627 non-null  datetime64[ns]
 7   End Station Name    85331 non-null  object        
 8   Bike Id             85627 non-null  string        
 9   User Type           85627 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(3), string(4)
memory usage: 7.2+ MB


## Calculate distance and average speed for each trip

Not a very precise calcluation as users were not constantly cycling, but we can still use it as an indicator; for the records that reveal unusual patterns (such as trip duration over 24 hr), we'll discuss after the calculation.

First, we need to load the geo info of each station

In [11]:
r = requests.get('https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information')

bikeshare_stations = pd.DataFrame(json.loads(r.content)['data']['stations'])[['station_id', 'name', 'lat', 'lon']].astype({
    'station_id': 'string',
})

bikeshare_stations

,station_id,name,lat,lon
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954
1,7001,Wellesley Station Green P,43.664964,-79.383550
2,7002,St. George St / Bloor St W,43.667333,-79.399429
3,7003,Madison Ave / Bloor St W,43.667158,-79.402761
4,7004,University Ave / Elm St,43.656518,-79.389099
...,...,...,...,...
612,7686,Lower Jarvis / The Esplanade,43.647821,-79.370472
613,7687,Bloor / Gladstone - SMART,43.660202,-79.434273
614,7688,High Park Blvd / Parkside Dr,43.644887,-79.456412
615,7689,20 Eaton,43.680012,-79.344033


Back to the bike share trips, we define route with start and end station ID

In [12]:
df01_v1["routes"] = df01_v1["Start Station Id"].str.cat("-" + df01_v1["End Station Id"])


Extract the unique routes

In [40]:
routes = df01_v1[['Start Station Id', 'End Station Id', "routes"]].drop_duplicates()

Match start and end coordinates

In [41]:
routes = routes.merge(bikeshare_stations, how='inner', left_on='Start Station Id', right_on='station_id') \
       .merge(bikeshare_stations, how='inner', left_on='End Station Id', right_on='station_id', suffixes=['_start', '_end']) \
       .drop_duplicates()

Calculate distance (use the distance function in geopy)

In [42]:
routes['distance'] = routes.apply(lambda x: distance.distance((x['lat_start'], x['lon_start']), (x['lat_end'], x['lon_end'])).m, axis=1)

In [43]:
routes_count = df01_v1.groupby("routes")['Trip Id'].count()
routes_count = routes_count.reset_index()

In [44]:
routes = routes.merge(routes_count[['routes', 'Trip Id']], how='left', on='routes')

,Start Station Id,End Station Id,routes,station_id_start,name_start,lat_start,lon_start,station_id_end,name_end,lat_end,lon_end,distance,Trip Id
0,7021,7164,7021-7164,7021,Bay St / Albert St,43.653264,-79.382458,7164,Gould St / Yonge St (Ryerson University),43.657424,-79.381019,476.553315,1
1,7006,7164,7006-7164,7006,Bay St / College St (East Side),43.660439,-79.385525,7164,Gould St / Yonge St (Ryerson University),43.657424,-79.381019,494.293898,1
2,7014,7164,7014-7164,7014,Sherbourne St / Carlton St (Allan Gardens),43.663102,-79.373181,7164,Gould St / Yonge St (Ryerson University),43.657424,-79.381019,893.139950,2
3,7053,7164,7053-7164,7053,Metro Hall Plaza,43.645859,-79.387830,7164,Gould St / Yonge St (Ryerson University),43.657424,-79.381019,1397.488587,1
4,7257,7164,7257-7164,7257,Dundas St W / St. Patrick St,43.654517,-79.389532,7164,Gould St / Yonge St (Ryerson University),43.657424,-79.381019,758.834737,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34978,7625,7625,7625-7625,7625,Yonge St / Lawrence Ave,43.725467,-79.401154,7625,Yonge St / Lawrence Ave,43.725467,-79.401154,0.000000,13
34979,7613,7625,7613-7625,7613,Livingston Rd (Highland Creek Trail),43.756168,-79.202669,7625,Yonge St / Lawrence Ave,43.725467,-79.401154,16348.698705,1
34980,7622,7621,7622-7621,7622,Marie Curtis Park,43.588077,-79.543207,7621,Long Branch GO Station,43.591561,-79.545827,441.111643,2
34981,7621,7621,7621-7621,7621,Long Branch GO Station,43.591561,-79.545827,7621,Long Branch GO Station,43.591561,-79.545827,0.000000,3


In [45]:
routes = routes.rename({'Trip Id': 'Count'}, axis='columns')


,Start Station Id,End Station Id,routes,station_id_start,name_start,lat_start,lon_start,station_id_end,name_end,lat_end,lon_end,distance,Count
0,7021,7164,7021-7164,7021,Bay St / Albert St,43.653264,-79.382458,7164,Gould St / Yonge St (Ryerson University),43.657424,-79.381019,476.553315,1
1,7006,7164,7006-7164,7006,Bay St / College St (East Side),43.660439,-79.385525,7164,Gould St / Yonge St (Ryerson University),43.657424,-79.381019,494.293898,1
2,7014,7164,7014-7164,7014,Sherbourne St / Carlton St (Allan Gardens),43.663102,-79.373181,7164,Gould St / Yonge St (Ryerson University),43.657424,-79.381019,893.139950,2
3,7053,7164,7053-7164,7053,Metro Hall Plaza,43.645859,-79.387830,7164,Gould St / Yonge St (Ryerson University),43.657424,-79.381019,1397.488587,1
4,7257,7164,7257-7164,7257,Dundas St W / St. Patrick St,43.654517,-79.389532,7164,Gould St / Yonge St (Ryerson University),43.657424,-79.381019,758.834737,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34978,7625,7625,7625-7625,7625,Yonge St / Lawrence Ave,43.725467,-79.401154,7625,Yonge St / Lawrence Ave,43.725467,-79.401154,0.000000,13
34979,7613,7625,7613-7625,7613,Livingston Rd (Highland Creek Trail),43.756168,-79.202669,7625,Yonge St / Lawrence Ave,43.725467,-79.401154,16348.698705,1
34980,7622,7621,7622-7621,7622,Marie Curtis Park,43.588077,-79.543207,7621,Long Branch GO Station,43.591561,-79.545827,441.111643,2
34981,7621,7621,7621-7621,7621,Long Branch GO Station,43.591561,-79.545827,7621,Long Branch GO Station,43.591561,-79.545827,0.000000,3


In [47]:
routes["Count"].describe()

count    34983.000000
mean         2.360404
std          2.977792
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max        121.000000
Name: Count, dtype: float64

Merge the distance calculation with the main DataFrame

In [16]:
df01_v2 = df01_v1.merge(routes[['routes', 'distance']], how='left', on='routes')
#df = df[[x for x in df.columns if not 'From' in x and not 'To' in x and x != 'Route Id']]
df01_v2

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type,routes,distance
0,10644218,1315,7021,2021-01-01 00:04:00,Bay St / Albert St,7164,2021-01-01 00:26:00,Gould St / Yonge St (Ryerson University),6795,Annual Member,7021-7164,476.553315
1,10644220,396,7534,2021-01-01 00:07:00,Walnut Ave / Queen St W,7524,2021-01-01 00:13:00,Lisgar Park,4176,Casual Member,7534-7524,1099.138396
2,10644221,86,7162,2021-01-01 00:10:00,Hayter St / Laplante Ave,7006,2021-01-01 00:11:00,Bay St / College St (East Side),1814,Annual Member,7162-7006,141.384845
3,10644222,741,7003,2021-01-01 00:10:00,Madison Ave / Bloor St W,7272,2021-01-01 00:22:00,Yonge St / Dundonald St - SMART,198,Casual Member,7003-7272,1456.774345
4,10644223,2073,7562,2021-01-01 00:11:00,Priscilla Ave / Dundas St W - SMART,7562,2021-01-01 00:45:00,Priscilla Ave / Dundas St W - SMART,6688,Casual Member,7562-7562,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
85622,10748535,316,7059,2021-01-31 23:45:00,Front St W / Blue Jays Way,7542,2021-01-31 23:50:00,Queen St W / John St,4531,Annual Member,7059-7542,736.672949
85623,10748536,158,7272,2021-01-31 23:45:00,Yonge St / Dundonald St - SMART,7044,2021-01-31 23:48:00,Church St / Alexander St,6681,Annual Member,7272-7044,430.766917
85624,10748537,1245,7662,2021-01-31 23:46:00,NaN,7259,2021-02-01 00:07:00,Lower Spadina Ave / Lake Shore Blvd,96,Annual Member,7662-7259,3946.351009
85625,10748538,393,7409,2021-01-31 23:53:00,Isabella St / Church St,7113,2021-01-31 23:59:00,Parliament St / Aberdeen Ave,6214,Casual Member,7409-7113,1180.602111


Calculate average speed (meters/second)

In [17]:
df01_v2["avg_speed"] = df01_v2["distance"]/ (df01_v2["Trip  Duration"])
df01_v2

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type,routes,distance,avg_speed
0,10644218,1315,7021,2021-01-01 00:04:00,Bay St / Albert St,7164,2021-01-01 00:26:00,Gould St / Yonge St (Ryerson University),6795,Annual Member,7021-7164,476.553315,0.362398
1,10644220,396,7534,2021-01-01 00:07:00,Walnut Ave / Queen St W,7524,2021-01-01 00:13:00,Lisgar Park,4176,Casual Member,7534-7524,1099.138396,2.775602
2,10644221,86,7162,2021-01-01 00:10:00,Hayter St / Laplante Ave,7006,2021-01-01 00:11:00,Bay St / College St (East Side),1814,Annual Member,7162-7006,141.384845,1.644010
3,10644222,741,7003,2021-01-01 00:10:00,Madison Ave / Bloor St W,7272,2021-01-01 00:22:00,Yonge St / Dundonald St - SMART,198,Casual Member,7003-7272,1456.774345,1.965957
4,10644223,2073,7562,2021-01-01 00:11:00,Priscilla Ave / Dundas St W - SMART,7562,2021-01-01 00:45:00,Priscilla Ave / Dundas St W - SMART,6688,Casual Member,7562-7562,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85622,10748535,316,7059,2021-01-31 23:45:00,Front St W / Blue Jays Way,7542,2021-01-31 23:50:00,Queen St W / John St,4531,Annual Member,7059-7542,736.672949,2.331244
85623,10748536,158,7272,2021-01-31 23:45:00,Yonge St / Dundonald St - SMART,7044,2021-01-31 23:48:00,Church St / Alexander St,6681,Annual Member,7272-7044,430.766917,2.726373
85624,10748537,1245,7662,2021-01-31 23:46:00,NaN,7259,2021-02-01 00:07:00,Lower Spadina Ave / Lake Shore Blvd,96,Annual Member,7662-7259,3946.351009,3.169760
85625,10748538,393,7409,2021-01-31 23:53:00,Isabella St / Church St,7113,2021-01-31 23:59:00,Parliament St / Aberdeen Ave,6214,Casual Member,7409-7113,1180.602111,3.004077


In [18]:
df01_v2["avg_speed"].describe()

count    82574.000000
mean         5.232585
std         58.197258
min          0.000000
25%          1.906260
50%          2.568133
75%          3.167054
max       3040.636202
Name: avg_speed, dtype: float64

## Special cases

From data cleaning and calculation of routes and speed, we find some special cases:

- **Same start and end station**

Reasons could be the bike was not working and the user wanted to change one, or the user changed mind and shift to other transportation tools, etc. Such cases can be identified by short trip duration like 1 or 2 seconds.

Or most likely, the user made a roundtrip. Therefore we can't make use of the distance and speed data, and we have to analyze such data separately.

In [19]:
df01_v2[df01_v2["distance"] == 0]

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type,routes,distance,avg_speed
4,10644223,2073,7562,2021-01-01 00:11:00,Priscilla Ave / Dundas St W - SMART,7562,2021-01-01 00:45:00,Priscilla Ave / Dundas St W - SMART,6688,Casual Member,7562-7562,0.0,0.0
25,10644246,1461,7164,2021-01-01 00:26:00,Gould St / Yonge St (Ryerson University),7164,2021-01-01 00:50:00,Gould St / Yonge St (Ryerson University),6795,Annual Member,7164-7164,0.0,0.0
31,10644252,643,7448,2021-01-01 00:29:00,Goodwood Park Ct / Dentonia Park - SMART,7448,2021-01-01 00:40:00,Goodwood Park Ct / Dentonia Park - SMART,6316,Annual Member,7448-7448,0.0,0.0
82,10644309,704,7519,2021-01-01 01:06:00,Sheridan Ave / Dundas St W - SMART,7519,2021-01-01 01:17:00,Sheridan Ave / Dundas St W - SMART,5875,Annual Member,7519-7519,0.0,0.0
94,10644326,446,7037,2021-01-01 01:20:00,Bathurst St / Dundas St W,7037,2021-01-01 01:28:00,Bathurst St / Dundas St W,2158,Casual Member,7037-7037,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85492,10748318,899,7375,2021-01-31 20:11:00,Front St E / Scott St,7375,2021-01-31 20:26:00,Front St E / Scott St,4176,Annual Member,7375-7375,0.0,0.0
85494,10748320,498,7206,2021-01-31 20:12:00,Claremont St / Dundas St W,7206,2021-01-31 20:20:00,Claremont St / Dundas St W,4988,Casual Member,7206-7206,0.0,0.0
85506,10748337,602,7035,2021-01-31 20:24:00,Queen St W / Ossington Ave,7035,2021-01-31 20:34:00,Queen St W / Ossington Ave,6447,Annual Member,7035-7035,0.0,0.0
85514,10748349,516,7458,2021-01-31 20:30:00,Church St / Lombard St,7458,2021-01-31 20:39:00,Church St / Lombard St,2488,Casual Member,7458-7458,0.0,0.0


- **Long distance within short time**

Usually the cycling speed is around 10 m/s, and 50 m/s would be considered phenomenal. When we query the records with average speed above 30, the majority of their trip durations are 2. So such pattern might indicate system error. 

In [20]:
df01_v2[(df01_v2["avg_speed"] > 30) & (df01_v2["Trip  Duration"] == 2)]

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type,routes,distance,avg_speed
226,10644516,2,7168,2021-01-01 07:05:00,Queens Quay / Yonge St,7044,2021-01-01 07:05:00,Church St / Alexander St,6090,Annual Member,7168-7044,2485.448622,1242.724311
995,10645472,2,7109,2021-01-01 14:40:00,Seaton St / Dundas St E - SMART,7389,2021-01-01 14:40:00,College Park- Gerrard Entrance,6409,Annual Member,7109-7389,1123.151179,561.575589
1463,10645994,2,7179,2021-01-01 16:49:00,Bloor GO / UP Station (West Toronto Railpath),7173,2021-01-01 16:49:00,Cariboo St / Rail Path,2441,Casual Member,7179-7173,1113.942251,556.971126
1536,10646087,2,7327,2021-01-01 17:14:00,Davenport Rd / Oakwood Rd - SMART,7419,2021-01-01 17:14:00,Bloor St W / Huron St,3157,Annual Member,7327-7419,2807.986568,1403.993284
1947,10646760,2,7032,2021-01-02 10:37:00,Augusta Ave / Dundas St W,7551,2021-01-02 10:37:00,The Esplanade / Hahn Pl - SMART,1981,Annual Member,7032-7551,3022.549307,1511.274654
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79339,10741055,2,7324,2021-01-28 18:29:00,King St W / Charlotte St (West),7505,2021-01-28 18:29:00,Ontario St / Adelaide St E -SMART,6882,Casual Member,7324-7505,2351.188138,1175.594069
79948,10741850,2,7026,2021-01-29 08:35:00,Bay St / St. Joseph St,7162,2021-01-29 08:35:00,Hayter St / Laplante Ave,205,Annual Member,7026-7162,723.846652,361.923326
81741,10743917,2,7047,2021-01-29 22:19:00,University Ave / Gerrard St W (East Side),7158,2021-01-29 22:19:00,King St W / Stafford St,4069,Casual Member,7047-7158,2473.085200,1236.542600
81997,10744232,2,7177,2021-01-30 09:31:00,East Liberty St / Pirandello St,7451,2021-01-30 09:31:00,Western Battery Rd / Pirandello St - SMART,6620,Annual Member,7177-7451,168.941036,84.470518


- **Long trip duration**

There is a few trips that lasted for more than 12 hrs, so we may not consider the speed of these trips for relevant analysis.

In [21]:
df01_v2[df01_v2["Trip  Duration"] > 43200]

,Trip Id,Trip Duration,Start Station Id,Start Time,Start Station Name,End Station Id,End Time,End Station Name,Bike Id,User Type,routes,distance,avg_speed
1327,10645828,81579,7569,2021-01-01 16:11:00,Toronto Inukshuk Park,7473,2021-01-02 14:51:00,Adelaide St W / Strachan Ave,5370,Casual Member,7569-7473,1222.964906,0.014991
7933,10654134,72269,7023,2021-01-04 18:37:00,College St / Borden St,7522,2021-01-05 14:41:00,Wallace Ave / Pauline Ave - SMART,458,Annual Member,7023-7522,2833.953065,0.039214
8480,10654733,48914,7069,2021-01-04 22:01:00,Queen St W / Spadina Ave,7000,2021-01-05 11:36:00,Fort York Blvd / Capreol Ct,6259,Annual Member,7069-7000,1035.582566,0.021171
8604,10654879,44840,7046,2021-01-05 00:45:00,Niagara St / Richmond St W,7329,2021-01-05 13:12:00,Crawford St / Queen St W,4886,Casual Member,7046-7329,465.164928,0.010374
25099,10675339,146313,7207,2021-01-09 20:10:00,Dundas St W / Crawford St,7153,2021-01-11 12:48:00,Bloor St W / Christie St,6040,Casual Member,7207-7153,1581.820624,0.010811
31059,10682427,80411,7148,2021-01-11 20:37:00,King St W / Joe Shuster Way,7633,2021-01-12 18:57:00,Noble St / Queen St W,5206,Casual Member,7148-7633,590.088224,0.007338
54720,10710857,57259,7450,2021-01-18 18:35:00,Carlaw Ave / Danforth Ave - SMART,7338,2021-01-19 10:30:00,Logan Ave / Bain Ave,1107,Casual Member,7450-7338,695.661418,0.012149
60736,10718437,62230,7160,2021-01-20 20:51:00,King St W / Tecumseth St,7524,2021-01-21 14:08:00,Lisgar Park,4527,Annual Member,7160-7524,1494.160649,0.024010
63982,10722544,59649,7336,2021-01-21 21:25:00,Queen St E / Alton Av,7357,2021-01-22 13:59:00,Lake Shore Blvd E / Leslie St,917,Casual Member,7336-7357,533.080953,0.008937
65399,10724406,199838,7226,2021-01-22 13:36:00,Lake Shore Blvd W / The Boulevard Club,7222,2021-01-24 21:07:00,Sunnyside - Gus Ryder Pool,6345,Annual Member,7226-7222,805.819961,0.004032


## Research questions

After exploring the data, I came up with a few questions that the Bike Share Toronto and the public might be curious:

- How many trips did poeple in Toronto had with Bike Share Toronto in 2021? 
- Was the ridership affected by season, weather, COVID-19, festivals and holidays, etc.?
- What are the riders' behavior in terms of distance, speed, time of the day, day of the week, etc.?
- What are the most popular cycling routes? Average speed on those routes? Any changes through the season?
- Troubleshooting: which stations and bikes were frequently caught up in malfunctionings?

With these questions in mind, we need functions to generate desirable datasets for each month in 2021. 

In [22]:
def clean(df):
    df01 = df[df["Trip  Duration"] != 0]
    df01 = df01[df01["End Station Id"].notnull()]
    df01 = df01[df01["Bike Id"].notnull()]
    return df01

In [28]:
def conversion(df):
    df['Start Time'] = pd.to_datetime(df['Start Time'])
    df['End Time'] = pd.to_datetime(df['End Time'])
    df['Start Station Id'] = df['Start Station Id'].astype("string")
    df['End Station Id'] = df['End Station Id'].astype("int")
    df['End Station Id'] = df['End Station Id'].astype("string")
    df['Bike Id'] = df['Bike Id'].astype("int")
    df['Bike Id'] = df['Bike Id'].astype("string")
    return df

In [24]:
def calculate(df):
    df["routes"] = df["Start Station Id"].str.cat("-" + df["End Station Id"])
    
    routes = df[['Start Station Id', 'End Station Id', "routes"]].drop_duplicates()
    routes = routes.merge(bikeshare_stations, how='inner', left_on='Start Station Id', right_on='station_id') \
       .merge(bikeshare_stations, how='inner', left_on='End Station Id', right_on='station_id', suffixes=['_start', '_end']) \
       .drop_duplicates()
    routes['distance'] = routes.apply(lambda x: distance.distance((x['lat_start'], x['lon_start']), (x['lat_end'], x['lon_end'])).m, axis=1)
    
    df01 = df.merge(routes[['routes', 'distance']], how='left', on='routes')
    df01["avg_speed"] = df01["distance"]/ (df01["Trip  Duration"])
    df01 = df01[(df01["avg_speed"] < 30) & (df01["Trip  Duration"] != 2)]
    df01 = df01[df01["Trip  Duration"] < 43200]
    return df01

In [25]:
def process(input_file, output_file):
    df01 = pd.read_csv(input_file, encoding='cp1252')
    print(df01.shape)
    df01 = df01[df01["Trip  Duration"] != 0]
    df01 = df01[df01["End Station Id"].notnull()]
    df01 = df01[df01["Bike Id"].notnull()]
    #df01['Start Time'] = pd.to_datetime(df01['Start Time'])
    #df01['End Time'] = pd.to_datetime(df01['End Time'])
    #df01['Trip Id'] = df01['Trip Id'].astype("string")
    df01['Start Station Id'] = df01['Start Station Id'].astype("string")
    df01['End Station Id'] = df01['End Station Id'].astype("int")
    df01['End Station Id'] = df01['End Station Id'].astype("string")
    #df01['Bike Id'] = df01['Bike Id'].astype("int")
    #df01['Bike Id'] = df01['Bike Id'].astype("string")
    df01["routes"] = df01["Start Station Id"].str.cat("-" + df01["End Station Id"])
    
    routes = df01[['Start Station Id', 'End Station Id', "routes"]].drop_duplicates()
    routes = routes.merge(bikeshare_stations, how='inner', left_on='Start Station Id', right_on='station_id') \
       .merge(bikeshare_stations, how='inner', left_on='End Station Id', right_on='station_id', suffixes=['_start', '_end']) \
       .drop_duplicates()
    routes['distance'] = routes.apply(lambda x: distance.distance((x['lat_start'], x['lon_start']), (x['lat_end'], x['lon_end'])).m, axis=1)
    
    df02 = df01.merge(routes[['routes', 'distance']], how='left', on='routes')
    df02["avg_speed"] = df02["distance"]/ (df02["Trip  Duration"])
    df02 = df02[(df02["avg_speed"] < 30) & (df02["Trip  Duration"] != 2)]
    df02 = df02[df02["Trip  Duration"] < 43200]
    df02.to_csv(output_file)


In [26]:
#process("bike-share-ridership-2021-12.csv", "12_cleaned.csv")